## Spatial Data Analysis Code-Along

> Author: Matt Brems

### Install packages if you haven't already:

In [ ]:
!pip install pysal
!pip install palettable
!pip install folium

A common package that is used to do spatial data analysis/exploration is [geopandas](http://geopandas.org/). However, there are a lot of weird dependencies that may give people trouble, so we'll avoid using geopandas today. There's a separate notebook that'll allow you to do some work in it if you want to test it out!
- I recommend that you set up an environment to do this if you want to try out geopandas.

In [ ]:
import numpy as np
import pandas as pd
from matplotlib.pyplot import *
import statsmodels.api as sm

%matplotlib inline

### First, we'll read in the shapefile and explore it a bit.

> A shapefile is a **file** that can hold the **shape** of an object. More technically, it is type of file that more easily contains the boundaries of objects and attributes of objects (i.e. the center of the object, the geographic location of the object if that exists) and it interfaces well with various programs.

In [ ]:
shp_link = './us48_data/us48.shp'

In [ ]:
import pysal.lib.io

In [ ]:
us = pysal.lib.io.open(shp_link)

#### Let's check out the header.

> The **BBOX** refers to a "bounding box," which is basically the box you'd create if you were to click and drag the smallest possible rectangle over a shape that contains the entire shape.

#### Let's check out one individual observation - in this case, it'll be a U.S. state.

#### There are plenty of [attributes that a polygon can have](http://pysal.readthedocs.io/en/latest/library/cg/shapes.html#pysal.cg.shapes.Polygon).

In [ ]:
print("Left: " + str(us[0].bounding_box.left))
print("Right: " + str(us[0].bounding_box.right))
print("Upper: " + str(us[0].bounding_box.upper))
print("Lower: " + str(us[0].bounding_box.lower))

In [ ]:
us[0].bounding_box.right - us[0].bounding_box.left

In [ ]:
us[0].bounding_box.upper - us[0].bounding_box.lower

In [ ]:
(us[0].bounding_box.right - us[0].bounding_box.left) * (us[0].bounding_box.upper - us[0].bounding_box.lower)

#### Thanks to Tucker Allen (DSI-US-3 grad from NYC) for the following code allowing you to visualize states by their vertices of the shapefile:

In [ ]:
x = []
y = []
for i in us[0].vertices: # Change index of us (0-47) to see vertices' points
    x.append(i[0])
    y.append(i[1])
fig, ax = matplotlib.pyplot.subplots(figsize=(10,6))
matplotlib.pyplot.scatter(x, y)
matplotlib.pyplot.xlim(us.header['BBOX Xmin'] , us.header['BBOX Xmax']) # limits of U.S.
matplotlib.pyplot.ylim(us.header['BBOX Ymin'] , us.header['BBOX Ymax']) # limits of U.S.

#### Let's read in weights and see what they look like.

> Weights will often be given to you, but sometimes you may need to specify weights. Below, we'll see how a tuple and dictionary are used to store the weights. The first value in the tuple refers to the specific observation/area; the second value in the tuple provides the dictionary of connected areas and the associated weights.

In [ ]:
w = pysal.lib.io.open('./us48_data/states48.gal').read()

#### We detect if spatial dependence exists using Moran's I, a permutation test.

In [ ]:
import pysal.explore

In [ ]:
y = np.array(pysal.lib.io.open('./us48_data/usjoin.csv').by_col('2009'))

In [ ]:
moran_i = pysal.explore.esda.Moran()

In [ ]:
# observed Moran's I statistic

Remember that -1 indicates a perfect negative relationship (chessboard) and +1 indicates perfect positive spatial relationship (two sides that were perfectly aligned color-wise).

In [ ]:
# z-score

In [ ]:
# p-value

Given our small $p$-value, we reject $H_0$ of no spatial association! Thus, **there is significant evidence to suggest that there is some spatial relationship in this 2009 median income data**.
> Note that we're not saying that the only influence on income is spatial. (That would be silly.) We're simply saying that, in looking at how our data are spatially organized, there appears to be some effect of space on median income in 2009.

#### Let's build a model taking space into account!

In [ ]:
y

In [ ]:
X = np.array(pysal.lib.io.open('./us48_data/usjoin.csv').by_col('2008'))

In [ ]:
y.resize(len(y), 1)

In [ ]:
X.resize(len(X), 1)

In [ ]:
y.shape, X.shape

In [ ]:
import pysal.model.spreg

In [ ]:
spatial_lm = pysal.model.spreg.OLS(y, X, w = None)

In [ ]:
spatial_lm_weights = pysal.model.spreg.OLS(y,
                                  X,
                                  w = w,
                                  spat_diag = True,
                                  moran = True)

In [ ]:
spatial_lm_weights = pysal.model.spreg.GM_Lag(y, X, w = w, spat_diag = True)

Spatial Auto-regressive Model:
$$
Y_i = \beta_0 + \rho \sum_{j} W_{ij}Y_j
$$

The model we fit:
$$
Y_i = \beta_0 + \beta_1[\text{2008 income}]_i + \rho \sum_{j} W_{ij}Y_j
$$

### Suppose want to generate our own weights!

In [ ]:
import pysal.lib

In [ ]:
wt = 